# 线性回归(使用自定义函数)

In [ ]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

# 1.生成合成数据集
def synthetic_data(w,b,num_exaples):
    """生成 y = Xw + b + 噪声"""
    X = torch.normal(0,1,(num_exaples,len(w)))
    print("X.shape:",X.shape)
    y = torch.matmul(X,w) + b
    print("y.shape:",y.shape)
    y += torch.normal(0,0.01,y.shape)
    print("y.shape:",y.shape)
    return X, y.reshape((-1,1))

true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print("features.shape:",features.shape)
print("labels.shape:",labels.shape)

In [ ]:
d2l.set_figsize()
d2l.plt.scatter(features[:,1].detach().numpy(),labels.detach().numpy(),1)  # 只有detach后才能转到numpy里面去     

In [ ]:
# 数据集迭代读取
# data_iter函数接收批量大小、特征矩阵和标签向量作为输入，生成大小为batch_size的小批量
def data_iter(batch_size,features,labels):
    num_examples = len(features)  # 样本个数
    indices = list(range(num_examples)) # 样本索引
    # 这些样本是随即读取的，没有特定的顺序
    random.shuffle(indices) # 把索引随即打乱
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size,num_examples)]) # 当i+batch_size超出时，取num_examples         
        yield features[batch_indices], labels[batch_indices] # 获得随即顺序的特征，及对应的标签
        
batch_size = 10
for X,y in data_iter(batch_size, features, labels):
    print(X, '\n', y) # 取一个批次后，就break跳出了
    break

In [ ]:
# 定义初始化模型参数
w = torch.normal(0,0.01,size=(2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

# 定义模型
def linreg(X,w,b):
    """线性回归模型"""
    return torch.matmul(X,w)+b

# 定义损失函数
def squared_loss(y_hat,y):
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape))**2/2 # 将y统一成与y_hat一样同尺寸   

# 定义优化算法
def sgd(params,lr,batch_size):
    """小批量随即梯度下降"""
    with torch.no_grad(): # 不要产生梯度计算，减少内存消耗
        for param in params: # 每个参数进行遍历
            param -= lr * param.grad / batch_size # 每个参数进行更新，损失函数没有求均值，所以这里除以 batch_size 求了均值。由于乘法的线性关系，这里除以放在loss的除以是等价的。                          
            param.grad.zero_() # 每个参数的梯度清零

# 训练过程
lr = 0.03
num_epochs = 3
net = linreg # 这里用线性模型，这样写是很方便net赋予其他模型，只需要改一处，不需要下面所有网络模型名称都改
loss = squared_loss

# 训练过程
for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l = loss(net(X,w,b),y) # x和y的小批量损失
        # 因为l是形状是(batch_size,1)，而不是一个标量。l中所有元素被加到一起
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w,b],lr,batch_size) #使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features,w,b),labels)
        print(f'epoch{epoch+1},loss{float(train_l.mean()):f}')   

# 比较真实参数和通过训练学到的参数来评估训练的成功程度
print(f'w的估计误差：{true_w-w.reshape(true_w.shape)}')
print(f'b的估计误差：{true_b-b}')

# 线性回归（使用PyTorch实现）

In [34]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l
from torch import nn

true_w = torch.tensor([2, -3.4])
true_b = 4.2
# 使用d2l.synthetic_data生成合成数据集
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

#  使用框架现有API读取数据
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)  # 将特征和标签打包成一个数据集
    return data.DataLoader(dataset, batch_size, shuffle=is_train)  # 返回数据迭代器

batch_size = 10
data_iter = load_array((features, labels), batch_size)
# print(next(iter(data_iter)))  # 取出一个批次的数据 iter(data_iter)返回一个迭代器，next(iter(data_iter))取出一个批次的数据

# 使用nn模块定义模型
class LinearNet(nn.Module):
    """线性回归模型"""
    def __init__(self):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(2, 1)  # 输入特征维度为2，输出维度为1

    def forward(self, X):
        return self.linear(X)
    
net = LinearNet()  # 实例化模型

# 初始化模型参数
net.linear.weight.data.normal_(0, 0.01) # 权重初始化为均值为0，标准差为0.01的正态分布
net.linear.bias.data.fill_(0)  # 偏置初始化为0

# 定义损失函数
loss = nn.MSELoss()  # 均方误差损失函数
# 定义优化算法
def init_optimizer(net, lr):
    """初始化优化器"""
    return torch.optim.SGD(net.parameters(), lr=lr)  # 使用随机梯度下降优化器

lr = 0.03
# 实例化优化器
trainer = init_optimizer(net, lr)  # 初始化优化器

num_epochs = 3
# 训练过程
for epoch in range(num_epochs):
    for X, y in data_iter:
        # 前向传播
        l = loss(net(X), y)  # 计算损失
        # 反向传播
        trainer.zero_grad()  # 清除梯度
        l.backward()  # 计算梯度
        trainer.step()  # 更新参数
    with torch.no_grad():
        train_l = loss(net(features), labels)  # 在整个训练集上计算损失
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')  # 打印每个epoch的平均损失



epoch 1, loss 0.000208
epoch 2, loss 0.000098
epoch 3, loss 0.000096
